In [1]:
import h5py
import numpy as np
data_Path='../data/TaxiBJ/BJ16_M32x32_T30_InOut.h5'
def del_incomp_date(data_Path):
    reader=h5py.File(data_Path,'r')
    data=reader['data'].value
    date=reader['date'].value
    reader.close()
    data=np.array(data)
    date=np.array(date)
    date_complete=[]##完整的天数
    data_complete=[]
    date_temp=[]##用于统计一天是否有48个时间戳
    data_temp=[]
    date_str=date[0][0:8]
    for i in range(len(date)):
        if date[i][0:8]==date_str:
            date_temp.append(date[i])
            data_temp.append(data[i])
        else :
            if len(date_temp)==48:
                date_complete.extend(date_temp)
                data_complete.extend(data_temp)
            date_temp.clear()
            data_temp.clear()
            date_str=date[i][0:8]
            date_temp.append(date[i])
            data_temp.append(data[i])
    print(len(date_complete),len(data_complete))
    return data_complete,date_complete
data,date = del_incomp_date(data_Path)
close_data=[]
period_data=[]
trend_data=[]
Y=data[1008:]
time_stamps=[]
close=[i for i in range(1,4)]
period=[48*j for j in range(1,4)]
trend=[7*48*j for j in range(1,4)]
close_temp=np.array([])
period_temp=np.array([])
trend_temp=np.array([])
data=np.array(data)
for i in range(1008,(len(data))):
    close_temp=np.vstack([data[i-j] for j in close])
    period_temp=np.vstack([data[i-j] for j in period])
    trend_temp=np.vstack([data[i-j] for j in trend])
    close_data.append(close_temp)
    period_data.append(period_temp)
    trend_data.append(trend_temp)
    time_stamps.append(date[i])
print(np.asarray(close_data).shape)
print(np.asarray(period_data).shape)
print(np.asarray(trend_data).shape)
print(np.asarray(time_stamps).shape)
print(np.asarray(Y).shape)

c:\users\hu\miniconda3\envs\pytorch11\lib\site-packages\h5py\_hl\dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


6624 6624
(5616, 6, 32, 32)
(5616, 6, 32, 32)
(5616, 6, 32, 32)
(5616,)
(5616, 2, 32, 32)


## 3 模型建立
### 3.1 基本Pytorch网络模型
    在代码实现模型之前，还是得先复习一下论文中的模型是怎样的。
    这篇论文的关键创新之处就是提出了“Spatio-Temporal Residual Networks”，基于时间 、空间的残差网络模型。
    对于close、period、trend三条分支来说，其实他们的模型是一样的，无非输入矩阵的维数可以不同而已。他们的模型有两个组成部分：
1. convolution 卷积层，将城市的车流信息看成是一张图片，这个部分获取了车流信息的位置联系，卷积核为3x3，最后以一个tanh函数结尾
2. residual unit 残差块，可以训练出非常深的卷积神经网络，在这个模型中，一个残差块被定义为ReLU->Conv->ReLU->Conv

In [2]:
from torch import  nn
import torch as t
from torch.nn import  functional as F
import torchvision.transforms as transforms

    首先下面的代码定义了一个残差块，其中用到了2维卷积、Relu.
    这个项目中输入Channel=6，Width=Height=32，输出Channel=2，Width=Height=32
    根据输入输出公式
![shape](img/conv-shape.jpg)

> class torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)

    因为要保持输出还是32x32，选择padding=1，kernel-size=3，stride=1

In [3]:
close_data=np.asarray(close_data)
close_data=np.float32(close_data)
print(close_data.shape)

(5616, 6, 32, 32)


In [4]:
class ResUnit(nn.Module):
    def __init__(self,in_flow,out_flow,stride=1,shortcut=True):
        super(ResUnit,self).__init__()
        self.left=nn.Sequential(
                nn.Conv2d(in_flow,out_flow,kernel_size=3,stride=1,padding=1,bias=False),
                nn.ReLU(inplace=True),
                nn.BatchNorm2d(out_flow),
                nn.Conv2d(out_flow,out_flow,kernel_size=3,stride=1,padding=1,bias=False),
                nn.BatchNorm2d(out_flow)).float()
        self.right=shortcut
    def forward(self,x):
        out=self.left(x)
        residual=x
        out+=residual
        return F.relu(out)

In [5]:
res=ResUnit(6,2).float()

In [6]:
a=t.from_numpy(close_data[0])
a.unsqueeze_(0)
a.requires_grad_(True)
a.float()
print(a.dtype)

torch.float32


In [7]:
print(res)

ResUnit(
  (left): Sequential(
    (0): Conv2d(6, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU(inplace)
    (2): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(2, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


In [8]:
para=list(res.parameters())
print(len(para))
print(para[0].size())

6
torch.Size([2, 6, 3, 3])


In [9]:
out=res(a)
print(out)

RuntimeError: The size of tensor a (2) must match the size of tensor b (6) at non-singleton dimension 1

### 3.2 feature模块 
    之前我们只使用了流量数据，没有看过天气情况，现在来看一看 BJ_Meteorology.h5 这个模块

In [10]:
import h5py
import numpy as np
feature_Path='BJ_Meteorology.h5'
reader=h5py.File(feature_Path,'r')
for key in reader.keys():
    print(key,reader[key].shape,reader[key].dtype)
temperature=reader['Temperature'].value
weather=reader['Weather'].value
windspeed=reader['WindSpeed'].value
feature_date=reader['date'].value

OSError: Unable to open file (unable to open file: name = 'BJ_Meteorology.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [11]:
print(feature_date[0:10])

[b'2013020105' b'2013020106' b'2013020107' b'2013020108' b'2013020109'
 b'2013020110' b'2013020111' b'2013020112' b'2013020113' b'2013020114']


    看来这个文件里有4个部分，温度、天气、风速和时间戳。

In [12]:
ws=[]
wr=[]
te=[]
M=dict()
for i, slot in enumerate(date):
    M[slot] = i
for i in time_stamps:
    feature_index=M[i]-1
    ws.append([windspeed[feature_index]])
    wr.append(weather[feature_index])
    te.append([temperature[feature_index]])
ws=np.array(ws)
wr=np.array(wr)
te=np.array(te)
ws=1.*(ws-ws.min())/(ws.max()-ws.min())
te=1.*(te-te.min())/(te.max()-te.min())
print(ws.ndim,wr.ndim,te.ndim)

2 2 2


In [13]:
merge_data=np.hstack([wr,ws,te])
print(merge_data.shape)

(5616, 19)


In [14]:
class Ext(nn.Module):
    def __init__(self,ext_dim):
        super(Ext,self).__init__()
        self.left=nn.Sequential(
                nn.Linear(in_features=ext_dim,out_features=10),
                nn.ReLU(inplace=True),
                nn.Linear(in_features=10,out_features=2*32*32),
                nn.ReLU(inplace=True))
    def forward(self,x):
        out=self.left(x)
        out=out.resize(2,32,32)
        return out

In [15]:
import torch
model=Ext(19)
a=torch.tensor(merge_data[0])
a=np.float32(a)
a=torch.tensor(a)
print(a.dtype)
out1=model(a)
print(out1)

torch.float32
tensor([[[ 0.0000,  0.1968,  0.0000,  ...,  0.0263,  0.3032,  0.0000],
         [ 0.0000,  0.0000,  0.2061,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.1103,  0.2472,  0.0000,  ...,  0.0000,  0.0000,  0.2043],
         ...,
         [ 0.0000,  0.2538,  0.0000,  ...,  0.0000,  0.1619,  0.0000],
         [ 0.2141,  0.2319,  0.1209,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.2215,  0.2876,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.2629,  0.0000,  0.0231,  ...,  0.1065,  0.1766,  0.1736],
         [ 0.2016,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0062],
         [ 0.0552,  0.2549,  0.0000,  ...,  0.1095,  0.0000,  0.2663],
         ...,
         [ 0.2389,  0.0000,  0.0000,  ...,  0.0000,  0.1941,  0.3339],
         [ 0.0000,  0.2345,  0.2899,  ...,  0.0105,  0.1450,  0.1565],
         [ 0.0000,  0.0983,  0.0672,  ...,  0.0195,  0.0000,  0.0000]]])


C:\Users\胡钧涵\AppData\Roaming\Python\Python36\site-packages\torch\tensor.py:255: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


In [16]:

m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
print(input.size(),output.size())

torch.Size([128, 20]) torch.Size([128, 30])


    如上我们简单的使用时间戳的数据流进行了多次计算，接下来，再来看看多个输入时如何fuse在一起的
### 3.3 输出之fuse
   ![fuse](img/fuse.jpg)
        
       如论文所示，每个输入要用一个系数哈达玛积之后加起来。
       那啥是Hadamard积呢，wiki告诉我
   ![hadamard](img/hadamard.jpg)

In [17]:
c=t.randn(2,2)
b=t.randn(2,2)
print(c)
print(b)

tensor([[ 0.4988,  0.9987],
        [ 0.5203, -0.0643]])
tensor([[ 1.1245, -0.1687],
        [-0.8883,  0.4992]])


In [18]:
d=t.mul(b,c)
print(d)

tensor([[ 0.5609, -0.1685],
        [-0.4622, -0.0321]])


    显然，torch.mul干的就是hadamard积

In [19]:
c_conf=(3, 2, 32, 32)
p_conf=(3, 2, 32, 32)
t_conf=(3, 2, 32, 32)
for conf in [c_conf, p_conf, t_conf]:
    print(conf)

(3, 2, 32, 32)
(3, 2, 32, 32)
(3, 2, 32, 32)


### 3.4 定义Loss function
    通过我们定义的网络输出的值要和预测值比较，计算损失函数，然后再反向传播，优化参数。
    

In [20]:
close_data=t.from_numpy(close_data)

In [21]:
data=np.float32(data)
data=t.from_numpy(data)
criterion=nn.MSELoss()
loss=criterion(out1,data[0])

In [22]:
print(loss)

tensor(308.8104)


In [23]:
print(data[0])

tensor([[[  0.,   2.,   4.,  ...,   1.,   1.,   9.],
         [  0.,   4.,   5.,  ...,   2.,   2.,  20.],
         [  0.,   1.,   1.,  ...,   2.,  51.,  37.],
         ...,
         [  1.,   1.,   3.,  ...,   0.,   0.,   0.],
         [  0.,   2.,   2.,  ...,   2.,   0.,   1.],
         [  0.,   2.,   1.,  ...,   1.,   2.,   2.]],

        [[  0.,   2.,   5.,  ...,   1.,   1.,   9.],
         [  0.,   4.,   6.,  ...,   2.,   2.,  21.],
         [  0.,   1.,   1.,  ...,   2.,  50.,  39.],
         ...,
         [  1.,   1.,   3.,  ...,   0.,   0.,   0.],
         [  0.,   2.,   2.,  ...,   2.,   0.,   1.],
         [  0.,   2.,   1.,  ...,   1.,   2.,   1.]]])


In [24]:

import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   
loss.backward()# zero the gradient buffers
optimizer.step()

In [25]:
out=model(a)
print(out)

tensor([[[ 0.0000,  0.1973,  0.0000,  ...,  0.0256,  0.3030,  0.0000],
         [ 0.0000,  0.0000,  0.2066,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.1102,  0.2469,  0.0000,  ...,  0.0000,  0.0000,  0.2050],
         ...,
         [ 0.0000,  0.2534,  0.0000,  ...,  0.0000,  0.1620,  0.0000],
         [ 0.2142,  0.2321,  0.1210,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.2216,  0.2878,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.2632,  0.0000,  0.0229,  ...,  0.1066,  0.1768,  0.1732],
         [ 0.2012,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0067],
         [ 0.0550,  0.2550,  0.0000,  ...,  0.1093,  0.0000,  0.2666],
         ...,
         [ 0.2387,  0.0000,  0.0000,  ...,  0.0000,  0.1939,  0.3336],
         [ 0.0000,  0.2351,  0.2897,  ...,  0.0111,  0.1448,  0.1566],
         [ 0.0000,  0.0981,  0.0669,  ...,  0.0190,  0.0000,  0.0000]]])


C:\Users\胡钧涵\AppData\Roaming\Python\Python36\site-packages\torch\tensor.py:255: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
